In [1]:
# Libraries
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
from PIL import Image
import matplotlib.pyplot as plt
from scipy.optimize import brute
%load_ext autoreload
%autoreload 2

### Fill in files and options

In [2]:
n_gfps = 120
brightness = 34 # 34 for egfp, 93 for mneon, 54 for sfgfp
#spots_folder = '/Volumes/stadler_5tb/old-red/2022-06-02/20220602-120mer-spots'
spots_folder = '/Volumes/stadler_5tb/old-red/2022-05-31/20220531-beads-1pct'
psf_pkl_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220614_insitu_nm100x50x50_px27x25x25_normed.pkl'
psf_dims = (100, 50, 50)
fitsize = (7,7)

In [3]:
# Functions

def load_tif_folder(folderpath):
    """Load tif files from folder."""
    ims = []
    for f in os.listdir(folderpath):
        if (f[0] != '.') and (f[-3:] == 'tif'):
            im = Image.open(os.path.join(folderpath, f))
            ims.append(np.array(im))
        else:
            print ('Skipping ' + f + '...')
    return ims

def center(ims, size=(8,8)):
    """Center box of indicated size at the center of spots.
    Center defined as max after gaussian smoothing (works well).
    """
    def inbounds(max_i, max_j, im_shape, size):
        if (max_i - size[0]) < 0:
            return False
        if (max_i + size[0] + 1) > (im_shape[0]):
            return False
        if (max_j - size[1]) < 0:
            return False
        if (max_j + size[1] + 1) > (im_shape[1]):
            return False
        return True

    ims_centered = []
    for im in ims:
        sm = ndi.gaussian_filter(im, 1.2)
        wheremax = np.where(sm == np.max(sm))
        max_i, max_j = wheremax[0][0], wheremax[1][0]
        if inbounds(max_i, max_j, im.shape, size):
            im_centered = im[
                (max_i - size[0]):(max_i + size[0] + 1), 
                (max_j - size[1]):(max_j + size[1] + 1)
                ]
            ims_centered.append(im_centered)
    return np.array(ims_centered)

def psf_match(psf, size):
    """Extract 2d slice of PSF which consists of the middle slice
    in Z and a size in xy matched to spot images."""
    psf_midslice = psf[int(psf.shape[0] / 2)]
    ij_midpoint = int(psf.shape[1] / 2)
    start = ij_midpoint - size[0] 
    stop = ij_midpoint + size[0] + 1
    psf_matched = psf_midslice[start:stop, start:stop]
    return psf_matched

def fit_spots_psf(spots, psf):
    """Use brute force optimization to determine the scalar multiple to best
    match the psf to the spot kernel."""
    def f(x, spots, psf):
        diff = abs(spots - (x * psf))
        return np.sum(diff)

    opt = brute(f, [slice(0,20000, 10)], args=(spots, psf), full_output=True)
    fit_x = opt[0][0]
    print(fit_x)
    return fit_x

def calc_nm_fromvol(im, lmask, label, im_dims, psf_dims, kernel_1gfp, brightness):
    """Calculate concentration from a volume image (Z stack)."""
    pix = im[lmask == label]
    sum_ = np.sum(pix)
    numpix = len(pix)
    kernel_1gfp_rescaled = ndi.zoom(kernel_1gfp, np.divide(psf_dims, im_dims))
    num_molecules = sum_ / np.sum(kernel_1gfp_rescaled)
    L = numpix * np.product(im_dims) * 1e-27 * 1000 # volume in liters
    mols = num_molecules / 6.022e23
    M = mols / L
    nM = M / 1e-9 
    nM = nM * (34 / brightness)
    print (nM)

def get_conc_uniform(kernel_1gfp, mean_signal, psf_dims, im_dims, brightness):
    """Calculate concentration assuming a uniform signal (measuring mean
    signal intensity and assuming it is uniform).
    
    Note: I thought it was conceptually simpler to work this as a 100x100x100
    cube but it's mathematically redundant. It was just easier for me to
    think about for some reason.
    """
    # Rescale kernel using interpolation to match image dimensions.
    kernel_1gfp_rescaled = ndi.zoom(kernel_1gfp, np.divide(psf_dims, im_dims))

    # 100x100x100 box
    sum_ = mean_signal * (100 ** 3)
    num_molecules = sum_ / np.sum(kernel_1gfp_rescaled)
    L = (100 ** 3) * np.product(im_dims) * 1e-27 * 1000 # volume in liters
    mols = num_molecules / 6.022e23
    M = mols / L
    nM = M / 1e-9 
    nM = nM * (34 / brightness)
    print (nM)

### Load, center, and inspect spot mean.

In [4]:
ims = load_tif_folder(spots_folder)
spots = center(ims, size=fitsize).mean(axis=0)
fm.viewer(spots)

interactive(children=(Dropdown(description='Color', index=10, options=('viridis', 'plasma', 'Gators', 'prism',…

### Fit PSF to mean spot image.

In [5]:
psf = fm.load_pickle(psf_pkl_file)
psf_matched = psf_match(psf, fitsize)

fit_scalar = fit_spots_psf(spots, psf_matched)    
kernel_1gfp = psf * fit_scalar / n_gfps


12968.854616165161


### Get concentrations for 3D volumes

In [55]:
# Load image.
im_file = '/Volumes/stadler_5tb/old-red/2022-05-31/rpb1-measurement-1pct-vol-em2-02.czi'
im_dims = (250, 50, 50)
im = fm.read_czi(im_file, swapaxes=False)[6:]
fm.viewer(im)

interactive(children=(Dropdown(description='Color', index=10, options=('viridis', 'plasma', 'Gators', 'prism',…

In [58]:
# Segment nuclei using simple smooth and theshold.
thresh = 2500
sm = ndi.gaussian_filter(im, 4)
mask = np.where(sm > thresh, 1, 0)
mask = ndi.morphology.binary_erosion(mask, np.ones((2,5,5)))
lmask,_ = ndi.label(mask)
fm.viewer(lmask.max(axis=0))


interactive(children=(Dropdown(description='Color', index=10, options=('viridis', 'plasma', 'Gators', 'prism',…

In [60]:
# Determine label of nucleus of interest.
label = 2
fm.viewer(np.where(lmask == label, im, 0).max(axis=0))

interactive(children=(Dropdown(description='Color', index=10, options=('viridis', 'plasma', 'Gators', 'prism',…

In [61]:
calc_nm_fromvol(im, lmask, label, im_dims, psf_dims, kernel_1gfp, brightness)

309.53289175201064


### Get concentrations from average signal (assuming uniform)

In [ ]:
# Note: the image dimensions don't actually matter (math cancels out).
im_dims = (250,50,50)
get_conc_uniform(kernel_1gfp, 5200, psf_dims, im_dims, brightness)


Zld: 3800-5200 mean: 100-130 nM (assume 70-130)

Rpb1: 

225 - 350
2022-05-31/rpb1-measurement-1pct-vol-em1-02.czi: 366
2022-05-31/rpb1-measurement-1pct-vol-em1-02.czi: 309
2022-05-31/rpb1-measurement-1pct-vol-em1-03.czi: 348,322
2022-05-31/rpb1-measurement-1pct-vol-em2-01.czi: 270
2022-05-31/rpb1-measurement-1pct-vol-em3-01.czi: 237,270, 292
2022-05-31/rpb1-measurement-1pct-vol-em3-02.czi: 274
